In [1]:
from google.cloud import storage
from firebase_admin import firestore, initialize_app

# Establish a connection to the Google Cloud Storage and Firestore
storage_client = storage.Client()
bucket = storage_client.bucket('website-content12345')
initialize_app()
db = firestore.client()

In [2]:
from datetime import datetime
# Okay let's insert some data into Firestore
doc_ref = db.collection('feed').document('content-log')

timestamp = datetime.now()
timestamp = timestamp.strftime("%Y-%m-%d %H:%M:%S")

data = {
    timestamp: {
        'title': 'Test Blog',
        'location': 'blogs/test-blog.md',
    }
}

# Upload it
doc_ref.set(data)

update_time {
  seconds: 1720282478
  nanos: 194105000
}

In [4]:
feed = db.collection('feed').document('content-log')
data = feed.get().to_dict()

# Sort by key (timestamp) desc
data = dict(sorted(data.items(), key=lambda item: item[0], reverse=True))
data

{'2024-07-06 17:14:38': {'location': 'blogs/test-blog.md'}}

In [9]:
for key, value in data.items():
    blob = bucket.blob(value['location'])
    md = blob.download_as_string().decode('utf-8')

# We can capture the section between --- and --- and use it as metadata
metadata = md.split('---')[1]
description = {}
for line in metadata.split('\n'):
    if line:
        split_line = line.split(':')
        key = split_line[0].strip()
        # Remove the quotes
        value = ":".join(split_line[1:]).strip()[1:-1]
        description[key.strip()] = value

description

{'title': 'Test Post',
 'author': 'Ed',
 'date': '2024-07-06',
 'tags': '"coding", "python"',
 'type': 'blog',
 'description': 'This is a blog post about coding in Python.',
 'thumbnail': 'images/awesome-blog-post-thumbnail.jpg',
 'og_title': 'Awesome Blog Post',
 'og_description': 'An amazing blog post about coding in Python.',
 'og_image': 'images/awesome-blog-post-og.jpg'}